# 🌱 CloudGarden - Smart Plant Monitoring System## HW3 - Refactored Version**Features:**- Real-time IoT Dashboard- Plant Disease Detection- RAG-powered Q&A (with improved search)- AI Chat with Database Persistence- Gamification System- Report Generation

## 📦 1. Dependencies Installation

In [ ]:
# Install all required packages
!pip install -q --upgrade gradio pandas matplotlib python-docx
!pip install -q --upgrade firebase-admin plotly gdown
!pip install -q cerebras-cloud-sdk
!pip install -q google-genai
!pip install -q scikit-learn  # For TF-IDF vectorization


## 📚 2. Imports

In [ ]:
# =============================================================================
# ALL IMPORTS - Organized by category
# =============================================================================

# --- Standard Library ---
import os
import re
import json
import time
import math
import random
import tempfile
import warnings
from datetime import datetime, timedelta, timezone
from zoneinfo import ZoneInfo
from collections import Counter, defaultdict
from typing import List, Dict, Tuple, Any, Optional
from urllib.parse import quote
from dataclasses import dataclass, field
from abc import ABC, abstractmethod
import hashlib

# --- Data Science ---
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- Plotly ---
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# --- ML/NLP ---
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

# --- Web ---
import requests
from bs4 import BeautifulSoup

# --- Gradio ---
import gradio as gr

# --- ML Model ---
from transformers import pipeline

# --- Firebase ---
import firebase_admin
from firebase_admin import credentials, db
import gdown

# --- Document Processing ---
from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Inches, Pt, RGBColor

# --- AI APIs ---
from cerebras.cloud.sdk import Cerebras
from google import genai
from google.genai import types

# --- Setup ---
warnings.filterwarnings('ignore')
nltk.download("stopwords", quiet=True)
nltk.download("punkt", quiet=True)

print("✅ All imports loaded successfully")


## ⚙️ 3. Configuration

In [ ]:
# =============================================================================
# CONFIGURATION - All settings in one place
# =============================================================================

@dataclass
class Config:
    """Central configuration for the entire application."""
    
    # --- Server Settings ---
    SERVER_URL: str = "https://server-cloud-v645.onrender.com/"
    FEED_NAME: str = "json"
    BATCH_LIMIT: int = 200
    
    # --- Firebase Settings ---
    FIREBASE_URL: str = "https://cloud-81451-default-rtdb.europe-west1.firebasedatabase.app/"
    FIREBASE_KEY_ID: str = "1ESnh8BIbGKrVEijA9nKNgNJNdD5kAaYC"
    FIREBASE_KEY_FILE: str = "firebase_key.json"
    
    # --- AI Models ---
    CEREBRAS_MODEL: str = "llama3.1-8b"
    GEMINI_MODEL: str = "gemini-2.5-flash"
    PLANT_MODEL: str = "linkanjarad/mobilenet_v2_1.0_224-plant-disease-identification"
    
    # --- Sensor Thresholds ---
    TEMP_MIN: float = 18.0
    TEMP_MAX: float = 32.0
    HUMIDITY_MIN: float = 35.0
    HUMIDITY_MAX: float = 75.0
    SOIL_MIN: float = 20.0
    SOIL_MAX: float = 60.0
    
    # --- Colors ---
    COLOR_TEMP: str = "#1f77b4"
    COLOR_HUMIDITY: str = "#ff7f0e"
    COLOR_SOIL: str = "#2ca02c"
    COLOR_OK: str = "#2ca02c"
    COLOR_WARN: str = "#ffbf00"
    COLOR_BAD: str = "#d62728"
    
    # --- Timezone ---
    TIMEZONE: str = "Asia/Jerusalem"
    
    # --- RAG Settings ---
    RAG_CHUNK_SIZE: int = 1000
    RAG_CHUNK_OVERLAP: int = 200
    RAG_TOP_K: int = 3
    BM25_K1: float = 1.5
    BM25_B: float = 0.75
    
    # --- Document URLs for RAG ---
    DOC_URLS: List[str] = field(default_factory=lambda: [
        "https://doi.org/10.1038/s41598-025-20629-y",
        "https://doi.org/10.3389/fpls.2016.01419",
        "https://doi.org/10.1038/s41598-025-05102-0",
        "https://doi.org/10.1038/s41598-025-04758-y",
        "https://doi.org/10.2174/0118743315321139240627092707",
    ])


# Create global config instance
CONFIG = Config()

print("✅ Configuration loaded")
print(f"   Server: {CONFIG.SERVER_URL}")
print(f"   Firebase: {CONFIG.FIREBASE_URL}")


## 🔥 4. Firebase Service

In [ ]:
# =============================================================================
# FIREBASE SERVICE - All Firebase operations
# =============================================================================

class FirebaseService:
    """Centralized Firebase operations."""
    
    _instance = None
    _initialized = False
    
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance
    
    def initialize(self) -> bool:
        """Initialize Firebase connection."""
        if self._initialized:
            return True
            
        try:
            # Download credentials
            if os.path.exists(CONFIG.FIREBASE_KEY_FILE):
                os.remove(CONFIG.FIREBASE_KEY_FILE)
            
            url = f'https://drive.google.com/uc?id={CONFIG.FIREBASE_KEY_ID}'
            gdown.download(url, CONFIG.FIREBASE_KEY_FILE, quiet=True, fuzzy=True)
            
            # Initialize Firebase Admin
            if not firebase_admin._apps:
                firebase_admin.initialize_app(
                    credentials.Certificate(CONFIG.FIREBASE_KEY_FILE),
                    {'databaseURL': CONFIG.FIREBASE_URL}
                )
            
            self._initialized = True
            print("✅ Firebase initialized")
            return True
            
        except Exception as e:
            print(f"❌ Firebase initialization failed: {e}")
            return False
    
    def get(self, path: str) -> Any:
        """Get data from Firebase path."""
        try:
            return db.reference(path).get()
        except Exception as e:
            print(f"Firebase GET error ({path}): {e}")
            return None
    
    def set(self, path: str, data: Any) -> bool:
        """Set data at Firebase path."""
        try:
            db.reference(path).set(data)
            return True
        except Exception as e:
            print(f"Firebase SET error ({path}): {e}")
            return False
    
    def push(self, path: str, data: Any) -> Optional[str]:
        """Push data to Firebase path, returns key."""
        try:
            ref = db.reference(path).push(data)
            return ref.key
        except Exception as e:
            print(f"Firebase PUSH error ({path}): {e}")
            return None
    
    def update(self, path: str, data: Dict) -> bool:
        """Update data at Firebase path."""
        try:
            db.reference(path).update(data)
            return True
        except Exception as e:
            print(f"Firebase UPDATE error ({path}): {e}")
            return False
    
    def delete(self, path: str) -> bool:
        """Delete data at Firebase path."""
        try:
            db.reference(path).delete()
            return True
        except Exception as e:
            print(f"Firebase DELETE error ({path}): {e}")
            return False
    
    def http_get(self, path: str) -> Any:
        """Direct HTTP GET (for RAG index operations)."""
        url = f"{CONFIG.FIREBASE_URL.rstrip('/')}/{path}.json"
        try:
            r = requests.get(url, timeout=30)
            return r.json() if r.status_code == 200 else None
        except:
            return None
    
    def http_put(self, path: str, data: Any) -> bool:
        """Direct HTTP PUT (for RAG index operations)."""
        url = f"{CONFIG.FIREBASE_URL.rstrip('/')}/{path}.json"
        try:
            r = requests.put(url, json=data, timeout=30)
            return r.status_code == 200
        except:
            return False


# Create global instance
firebase = FirebaseService()
firebase.initialize()


## 🤖 5. AI API Clients

In [ ]:
# =============================================================================
# AI API CLIENTS - Cerebras and Gemini
# =============================================================================

class AIClients:
    """Manage AI API clients."""
    
    _cerebras_client = None
    _gemini_client = None
    
    @classmethod
    def get_cerebras(cls):
        """Get Cerebras client."""
        if cls._cerebras_client is None:
            api_key = os.environ.get("CEREBRAS_API_KEY", "csk-r8npfcy9jckcxcd98t4422mw99wx3ew89k4h3rrhdvy5ekde")
            os.environ["CEREBRAS_API_KEY"] = api_key
            cls._cerebras_client = Cerebras(api_key=api_key)
        return cls._cerebras_client
    
    @classmethod
    def get_gemini(cls):
        """Get Gemini client."""
        if cls._gemini_client is None:
            # Try multiple sources for API key
            api_key = None
            try:
                from google.colab import userdata
                api_key = userdata.get("GEMINI_API_KEY") or userdata.get("GOOGLE_API_KEY")
            except:
                pass
            
            if not api_key:
                api_key = os.environ.get("GEMINI_API_KEY") or os.environ.get("GOOGLE_API_KEY")
            
            if not api_key:
                raise RuntimeError("Missing GEMINI_API_KEY")
            
            cls._gemini_client = genai.Client(api_key=api_key)
        return cls._gemini_client
    
    @classmethod
    def generate_gemini(cls, prompt: str, temperature: float = 0.7, max_tokens: int = 1024) -> str:
        """Generate text using Gemini."""
        try:
            client = cls.get_gemini()
            response = client.models.generate_content(
                model=CONFIG.GEMINI_MODEL,
                contents=[types.Content(role="user", parts=[types.Part.from_text(text=prompt)])],
                config=types.GenerateContentConfig(
                    temperature=temperature,
                    max_output_tokens=max_tokens,
                ),
            )
            return (response.text or "").strip()
        except Exception as e:
            print(f"Gemini error: {e}")
            return ""
    
    @classmethod
    def generate_cerebras(cls, prompt: str, system: str = "") -> str:
        """Generate text using Cerebras."""
        try:
            client = cls.get_cerebras()
            messages = []
            if system:
                messages.append({"role": "system", "content": system})
            messages.append({"role": "user", "content": prompt})
            
            response = client.chat.completions.create(
                model=CONFIG.CEREBRAS_MODEL,
                messages=messages,
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"Cerebras error: {e}")
            return ""


# Load plant disease classifier
plant_classifier = pipeline("image-classification", model=CONFIG.PLANT_MODEL)
print("✅ AI clients ready")


## 📝 6. NLP Utilities

In [ ]:
# =============================================================================
# NLP UTILITIES - Text processing for RAG
# =============================================================================

class NLPProcessor:
    """NLP processing utilities for RAG."""
    
    def __init__(self):
        self.stemmer = PorterStemmer()
        self.stop_words = set(stopwords.words("english"))
        self._tfidf_vectorizer = None
        self._doc_vectors = None
    
    def tokenize(self, text: str) -> List[str]:
        """Tokenize text into words."""
        return re.findall(r"\w+", (text or "").lower())
    
    def preprocess(self, text: str) -> List[str]:
        """Full preprocessing: tokenize, remove stopwords, stem."""
        tokens = self.tokenize(text)
        tokens = [t for t in tokens if t not in self.stop_words and len(t) > 2]
        tokens = [self.stemmer.stem(t) for t in tokens]
        return tokens
    
    def preprocess_text(self, text: str) -> str:
        """Preprocess and join back to string."""
        return " ".join(self.preprocess(text))
    
    def chunk_text(self, text: str, chunk_size: int = None, overlap: int = None) -> List[str]:
        """Split text into overlapping chunks."""
        chunk_size = chunk_size or CONFIG.RAG_CHUNK_SIZE
        overlap = overlap or CONFIG.RAG_CHUNK_OVERLAP
        
        if not text or len(text) < chunk_size:
            return [text] if text else []
        
        chunks = []
        start = 0
        while start < len(text):
            end = start + chunk_size
            chunk = text[start:end]
            
            # Try to break at sentence boundary
            if end < len(text):
                last_period = chunk.rfind('.')
                if last_period > chunk_size // 2:
                    chunk = chunk[:last_period + 1]
                    end = start + last_period + 1
            
            chunks.append(chunk.strip())
            start = end - overlap
        
        return [c for c in chunks if c]
    
    def build_tfidf(self, documents: List[str]) -> None:
        """Build TF-IDF vectors for documents."""
        if not documents:
            return
        
        self._tfidf_vectorizer = TfidfVectorizer(
            preprocessor=self.preprocess_text,
            max_features=5000,
            ngram_range=(1, 2)
        )
        self._doc_vectors = self._tfidf_vectorizer.fit_transform(documents)
    
    def tfidf_search(self, query: str, top_k: int = 3) -> List[Tuple[int, float]]:
        """Search using TF-IDF similarity."""
        if self._tfidf_vectorizer is None or self._doc_vectors is None:
            return []
        
        query_vec = self._tfidf_vectorizer.transform([query])
        similarities = cosine_similarity(query_vec, self._doc_vectors).flatten()
        
        top_indices = similarities.argsort()[-top_k:][::-1]
        return [(int(idx), float(similarities[idx])) for idx in top_indices if similarities[idx] > 0]


# Create global NLP processor
nlp = NLPProcessor()
print("✅ NLP processor ready")


## 📊 7. Sensor Data Service

In [ ]:
# =============================================================================
# SENSOR DATA SERVICE - IoT data operations
# =============================================================================

class SensorDataService:
    """Handle all sensor data operations."""
    
    @staticmethod
    def fetch_from_server(feed: str, limit: int = 200) -> Optional[pd.DataFrame]:
        """Fetch sensor data from IoT server."""
        try:
            resp = requests.get(
                f"{CONFIG.SERVER_URL}/history",
                params={"feed": feed, "limit": limit},
                timeout=30
            )
            data = resp.json()
            
            if "data" not in data or not data["data"]:
                return None
            
            df = pd.DataFrame(data["data"])
            if "created_at" not in df.columns or "value" not in df.columns:
                return None
            
            df["created_at"] = pd.to_datetime(df["created_at"], errors="coerce", utc=True)
            df["value"] = pd.to_numeric(df["value"], errors="coerce")
            df = df.dropna(subset=["created_at", "value"]).sort_values("created_at")
            
            return df if not df.empty else None
            
        except Exception as e:
            print(f"Server fetch error: {e}")
            return None
    
    @staticmethod
    def load_from_firebase() -> pd.DataFrame:
        """Load all sensor data from Firebase."""
        data = firebase.get('/sensor_data')
        if not data:
            return pd.DataFrame()
        
        records = []
        for v in data.values():
            try:
                records.append({
                    'timestamp': pd.to_datetime(v['created_at']),
                    'temperature': float(v['temperature']),
                    'humidity': float(v['humidity']),
                    'soil': float(v['soil'])
                })
            except:
                continue
        
        if not records:
            return pd.DataFrame()
        
        df = pd.DataFrame(records)
        df = df.sort_values('timestamp').reset_index(drop=True)
        
        # Clip to valid ranges
        df['temperature'] = df['temperature'].clip(-50, 100)
        df['humidity'] = df['humidity'].clip(0, 100)
        df['soil'] = df['soil'].clip(0, 100)
        
        return df
    
    @staticmethod
    def sync_to_firebase() -> Tuple[str, int]:
        """Sync new data from server to Firebase."""
        msgs = ["Starting sync..."]
        
        # Get latest timestamp from Firebase
        try:
            latest_data = db.reference('/sensor_data').order_by_child('created_at').limit_to_last(1).get()
            latest_ts = list(latest_data.values())[0]['created_at'] if latest_data else None
        except:
            latest_ts = None
        
        msgs.append(f"Latest in DB: {latest_ts or 'None'}")
        
        # Fetch from server
        try:
            resp = requests.get(
                f"{CONFIG.SERVER_URL}/history",
                params={"feed": CONFIG.FEED_NAME, "limit": CONFIG.BATCH_LIMIT},
                timeout=180
            ).json()
        except Exception as e:
            return "\n".join(msgs + [f"Server error: {e}"]), 0
        
        if "data" not in resp:
            return "\n".join(msgs + ["No data from server"]), 0
        
        # Filter new records
        new_records = [s for s in resp["data"] if not latest_ts or s["created_at"] > latest_ts]
        
        if not new_records:
            return "\n".join(msgs + ["No new data"]), 0
        
        # Save to Firebase
        ref = db.reference('/sensor_data')
        saved = 0
        
        for sample in new_records:
            try:
                vals = json.loads(sample['value'])
                timestamp_key = sample['created_at'].replace(':', '-').replace('.', '-')
                
                ref.child(timestamp_key).set({
                    'created_at': sample['created_at'],
                    'temperature': max(-50, min(100, float(vals['temperature']))),
                    'humidity': max(0, min(100, float(vals['humidity']))),
                    'soil': max(0, min(100, float(vals['soil'])))
                })
                saved += 1
            except:
                continue
        
        return "\n".join(msgs + [f"Found {len(new_records)} new", f"Saved {saved}!"]), saved
    
    @staticmethod
    def check_plant_status(temp: float, humidity: float, soil: float) -> Tuple[str, str, List[str]]:
        """Check plant health based on sensor values."""
        issues = []
        warnings = []
        
        checks = [
            ("Temperature", temp, CONFIG.TEMP_MIN, CONFIG.TEMP_MAX, 1),
            ("Air humidity", humidity, CONFIG.HUMIDITY_MIN, CONFIG.HUMIDITY_MAX, 3),
            ("Soil moisture", soil, CONFIG.SOIL_MIN, CONFIG.SOIL_MAX, 3),
        ]
        
        for name, value, low, high, margin in checks:
            if not (low <= value <= high):
                issues.append(f"{name} out of range ({value:.1f})")
            elif value <= low + margin or value >= high - margin:
                warnings.append(f"{name} near limit ({value:.1f})")
        
        if issues:
            status = "🔴 Plant Status: Not OK"
            color = "bad"
        elif warnings:
            status = "🟡 Plant Status: Warning"
            color = "warn"
        else:
            status = "🟢 Plant Status: OK"
            color = "ok"
        
        details = issues + warnings if (issues or warnings) else ["All sensors within valid ranges"]
        
        return status, color, details


# Create global instance
sensor_service = SensorDataService()
print("✅ Sensor data service ready")


## 💬 8. Chat Service (with Database Persistence)

In [ ]:
# =============================================================================
# CHAT SERVICE - Conversations stored in Firebase
# =============================================================================

class ChatService:
    """Chat service with Firebase persistence for conversation history."""
    
    FIREBASE_PATH = "chat_conversations"
    
    def __init__(self):
        self.current_session_id: Optional[str] = None
        self.session_history: List[Tuple[str, str]] = []
    
    def _generate_session_id(self) -> str:
        """Generate unique session ID."""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        random_suffix = hashlib.md5(str(random.random()).encode()).hexdigest()[:6]
        return f"session_{timestamp}_{random_suffix}"
    
    def start_new_session(self) -> str:
        """Start a new chat session."""
        self.current_session_id = self._generate_session_id()
        self.session_history = []
        
        # Create session in Firebase
        session_data = {
            "created_at": datetime.now().isoformat(),
            "updated_at": datetime.now().isoformat(),
            "messages": []
        }
        firebase.set(f"{self.FIREBASE_PATH}/{self.current_session_id}", session_data)
        
        return self.current_session_id
    
    def load_session(self, session_id: str) -> bool:
        """Load existing session from Firebase."""
        data = firebase.get(f"{self.FIREBASE_PATH}/{session_id}")
        if not data:
            return False
        
        self.current_session_id = session_id
        messages = data.get("messages", [])
        
        # Reconstruct history as list of tuples
        self.session_history = []
        for msg in messages:
            if msg.get("role") == "user":
                user_msg = msg.get("content", "")
                # Find corresponding assistant message
                idx = messages.index(msg)
                if idx + 1 < len(messages) and messages[idx + 1].get("role") == "assistant":
                    assistant_msg = messages[idx + 1].get("content", "")
                    self.session_history.append((user_msg, assistant_msg))
        
        return True
    
    def get_all_sessions(self) -> List[Dict]:
        """Get list of all chat sessions."""
        data = firebase.get(self.FIREBASE_PATH)
        if not data:
            return []
        
        sessions = []
        for session_id, session_data in data.items():
            sessions.append({
                "id": session_id,
                "created_at": session_data.get("created_at", ""),
                "message_count": len(session_data.get("messages", [])),
            })
        
        # Sort by creation time, newest first
        sessions.sort(key=lambda x: x["created_at"], reverse=True)
        return sessions
    
    def _save_message(self, role: str, content: str) -> None:
        """Save a message to Firebase."""
        if not self.current_session_id:
            self.start_new_session()
        
        message = {
            "role": role,
            "content": content,
            "timestamp": datetime.now().isoformat()
        }
        
        # Get current messages
        path = f"{self.FIREBASE_PATH}/{self.current_session_id}"
        data = firebase.get(path) or {"messages": []}
        messages = data.get("messages", [])
        
        # Handle Firebase converting empty list to None
        if messages is None:
            messages = []
        
        messages.append(message)
        
        # Update session
        firebase.update(path, {
            "messages": messages,
            "updated_at": datetime.now().isoformat()
        })
    
    def chat(self, user_message: str, temperature: float = 0.7) -> Tuple[str, List[Tuple[str, str]]]:
        """Send message and get response, persisted to database."""
        user_message = (user_message or "").strip()
        if not user_message:
            return "", self.session_history
        
        # Ensure we have a session
        if not self.current_session_id:
            self.start_new_session()
        
        # Build conversation context
        contents = []
        for u, a in self.session_history:
            contents.append(types.Content(role="user", parts=[types.Part.from_text(text=u)]))
            contents.append(types.Content(role="model", parts=[types.Part.from_text(text=a)]))
        contents.append(types.Content(role="user", parts=[types.Part.from_text(text=user_message)]))
        
        # Generate response
        try:
            client = AIClients.get_gemini()
            response = client.models.generate_content(
                model=CONFIG.GEMINI_MODEL,
                contents=contents,
                config=types.GenerateContentConfig(
                    system_instruction="You are a helpful, friendly chatbot for a smart garden application. Answer clearly and helpfully.",
                    temperature=temperature,
                    max_output_tokens=512,
                ),
            )
            answer = (response.text or "").strip()
        except Exception as e:
            answer = f"Error generating response: {e}"
        
        if not answer:
            answer = "I couldn't generate an answer. Please try again."
        
        # Save to database
        self._save_message("user", user_message)
        self._save_message("assistant", answer)
        
        # Update local history
        self.session_history.append((user_message, answer))
        
        return answer, self.session_history
    
    def clear_session(self) -> List[Tuple[str, str]]:
        """Clear current session and start new one."""
        self.start_new_session()
        return []
    
    def delete_session(self, session_id: str) -> bool:
        """Delete a session from Firebase."""
        return firebase.delete(f"{self.FIREBASE_PATH}/{session_id}")
    
    def get_session_history_formatted(self) -> str:
        """Get formatted history for display."""
        if not self.session_history:
            return "No messages yet."
        
        lines = []
        for i, (user, assistant) in enumerate(self.session_history, 1):
            lines.append(f"[{i}] User: {user}")
            lines.append(f"    Assistant: {assistant}")
            lines.append("")
        
        return "\n".join(lines)


# Create global chat service
chat_service = ChatService()
print("✅ Chat service ready (with database persistence)")


## 📄 9. Document Fetcher (for RAG)

In [ ]:
# =============================================================================
# DOCUMENT FETCHER - Fetch and process documents for RAG
# =============================================================================

class DocumentFetcher:
    """Fetch and extract text from documents (HTML, PDF)."""
    
    HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
        "Accept-Language": "en-US,en;q=0.9",
    }
    
    # Sections to skip in academic papers
    SKIP_SECTIONS = {
        "references", "bibliography", "acknowledgements", "acknowledgments",
        "author information", "ethics declarations", "additional information",
        "supplementary information", "rights and permissions", "data availability"
    }
    
    @classmethod
    def fetch_html(cls, url: str, timeout: int = 25) -> Tuple[str, str, int]:
        """Fetch HTML from URL."""
        try:
            r = requests.get(url, headers=cls.HEADERS, timeout=timeout, allow_redirects=True)
            return r.text or "", r.url, r.status_code
        except Exception as e:
            return "", url, 0
    
    @classmethod
    def extract_text_from_html(cls, html: str) -> str:
        """Extract main text content from HTML."""
        if not html:
            return ""
        
        try:
            soup = BeautifulSoup(html, "lxml")
        except:
            soup = BeautifulSoup(html, "html.parser")
        
        # Remove unwanted elements
        for tag in soup(["script", "style", "noscript", "svg", "iframe", "nav", "footer"]):
            tag.decompose()
        
        # Try to find main content
        root = soup.find("main") or soup.find("article") or soup.find("div", class_="content") or soup
        
        # Extract title
        title = ""
        if soup.title and soup.title.string:
            title = soup.title.string.strip()
        
        # Extract description
        desc = ""
        meta = soup.find("meta", attrs={"name": "description"})
        if meta and meta.get("content"):
            desc = meta["content"].strip()
        
        # Extract body text
        chunks = []
        for el in root.find_all(["h1", "h2", "h3", "p", "li"]):
            text = el.get_text(" ", strip=True)
            if text and len(text) >= 30:
                # Skip reference sections
                text_lower = text.lower()
                if any(skip in text_lower for skip in cls.SKIP_SECTIONS):
                    continue
                chunks.append(text)
        
        # Remove duplicates while preserving order
        chunks = list(dict.fromkeys(chunks))
        
        # Build result
        parts = []
        if title:
            parts.append(f"TITLE: {title}")
        if desc:
            parts.append(f"DESCRIPTION: {desc}")
        if chunks:
            parts.append("\n".join(chunks))
        
        return "\n".join(parts).strip()
    
    @classmethod
    def normalize_doi(cls, url: str) -> str:
        """Extract DOI from URL."""
        s = (url or "").strip()
        s = s.replace("https://doi.org/", "").replace("http://doi.org/", "")
        return s.strip()
    
    @classmethod
    def fetch_semantic_scholar(cls, doi: str) -> Dict:
        """Fetch metadata from Semantic Scholar."""
        try:
            url = f"https://api.semanticscholar.org/graph/v1/paper/DOI:{doi}?fields=title,abstract,openAccessPdf"
            r = requests.get(url, timeout=15)
            return r.json() if r.status_code == 200 else {}
        except:
            return {}
    
    @classmethod
    def fetch_openalex(cls, doi: str) -> Dict:
        """Fetch metadata from OpenAlex."""
        try:
            url = f"https://api.openalex.org/works/https://doi.org/{doi}"
            r = requests.get(url, timeout=15)
            if r.status_code == 200:
                data = r.json()
                return {
                    "title": data.get("title", ""),
                    "abstract": data.get("abstract", ""),
                    "pdf_url": data.get("open_access", {}).get("oa_url", "")
                }
        except:
            pass
        return {}
    
    @classmethod
    def get_document_text(cls, url: str, min_chars: int = 500) -> str:
        """Get best available text for a document URL."""
        # Try HTML extraction first
        html, final_url, status = cls.fetch_html(url)
        text = cls.extract_text_from_html(html)
        
        if len(text) >= min_chars:
            return text
        
        # Try academic APIs for DOI URLs
        if "doi.org/" in url:
            doi = cls.normalize_doi(url)
            
            # Try Semantic Scholar
            ss = cls.fetch_semantic_scholar(doi)
            if ss:
                parts = []
                if ss.get("title"):
                    parts.append(f"TITLE: {ss['title']}")
                if ss.get("abstract"):
                    parts.append(f"ABSTRACT: {ss['abstract']}")
                
                api_text = "\n".join(parts)
                if len(api_text) > len(text):
                    text = api_text
            
            # Try OpenAlex
            if len(text) < min_chars:
                oa = cls.fetch_openalex(doi)
                if oa.get("abstract"):
                    parts = []
                    if oa.get("title"):
                        parts.append(f"TITLE: {oa['title']}")
                    parts.append(f"ABSTRACT: {oa['abstract']}")
                    api_text = "\n".join(parts)
                    if len(api_text) > len(text):
                        text = api_text
        
        return text


print("✅ Document fetcher ready")


## 🔍 10. RAG Service (Improved with TF-IDF + BM25)

In [ ]:
# =============================================================================
# RAG SERVICE - Improved retrieval with hybrid TF-IDF + BM25 ranking
# =============================================================================

class RAGService:
    """RAG service with improved search using TF-IDF + BM25 hybrid ranking."""
    
    FIREBASE_INDEX_PATH = "indexes/public_index"
    FIREBASE_DOCMAP_PATH = "indexes/doc_map"
    FIREBASE_DOCTEXT_PATH = "indexes/doc_text"
    
    def __init__(self):
        self.documents: Dict[int, str] = {}  # doc_id -> full text
        self.chunks: Dict[int, List[str]] = {}  # doc_id -> list of chunks
        self.all_chunks: List[Tuple[int, int, str]] = []  # (doc_id, chunk_id, text)
        self.doc_urls: Dict[int, str] = {}  # doc_id -> URL
        self.inverted_index: Dict[str, List[int]] = {}  # term -> doc_ids
        self.doc_lengths: Dict[int, int] = {}  # doc_id -> token count
        self.avg_doc_length: float = 0
        self.is_loaded: bool = False
        
        # TF-IDF components
        self._tfidf_vectorizer: Optional[TfidfVectorizer] = None
        self._chunk_vectors = None
    
    def build_index(self, urls: List[str], force_rebuild: bool = False) -> bool:
        """Build or load the RAG index."""
        
        # Try to load from Firebase first
        if not force_rebuild:
            if self._load_from_firebase():
                self._build_search_structures()
                print(f"✅ Loaded index from Firebase ({len(self.documents)} docs)")
                return True
        
        print("Building new index...")
        
        # Fetch documents
        for i, url in enumerate(urls):
            print(f"  Fetching doc {i+1}/{len(urls)}: {url[:50]}...")
            text = DocumentFetcher.get_document_text(url)
            self.documents[i] = text
            self.doc_urls[i] = url
            
            # Create chunks
            self.chunks[i] = nlp.chunk_text(text)
        
        # Build inverted index
        self._build_inverted_index()
        
        # Save to Firebase
        self._save_to_firebase()
        
        # Build search structures
        self._build_search_structures()
        
        self.is_loaded = True
        print(f"✅ Index built ({len(self.documents)} docs, {len(self.all_chunks)} chunks)")
        return True
    
    def _build_inverted_index(self) -> None:
        """Build inverted index for BM25."""
        self.inverted_index = defaultdict(set)
        total_length = 0
        
        for doc_id, text in self.documents.items():
            tokens = nlp.preprocess(text)
            self.doc_lengths[doc_id] = len(tokens)
            total_length += len(tokens)
            
            for term in set(tokens):
                self.inverted_index[term].add(doc_id)
        
        # Convert sets to lists
        self.inverted_index = {k: list(v) for k, v in self.inverted_index.items()}
        self.avg_doc_length = total_length / max(len(self.documents), 1)
    
    def _build_search_structures(self) -> None:
        """Build TF-IDF vectors for chunk-level search."""
        # Flatten chunks
        self.all_chunks = []
        for doc_id, chunks in self.chunks.items():
            for chunk_id, chunk in enumerate(chunks):
                self.all_chunks.append((doc_id, chunk_id, chunk))
        
        if not self.all_chunks:
            return
        
        # Build TF-IDF
        chunk_texts = [c[2] for c in self.all_chunks]
        self._tfidf_vectorizer = TfidfVectorizer(
            preprocessor=nlp.preprocess_text,
            max_features=5000,
            ngram_range=(1, 2),
            sublinear_tf=True
        )
        self._chunk_vectors = self._tfidf_vectorizer.fit_transform(chunk_texts)
        self.is_loaded = True
    
    def _load_from_firebase(self) -> bool:
        """Load index from Firebase."""
        try:
            doc_text = firebase.http_get(self.FIREBASE_DOCTEXT_PATH)
            doc_map = firebase.http_get(self.FIREBASE_DOCMAP_PATH)
            inv_index = firebase.http_get(self.FIREBASE_INDEX_PATH)
            
            if not doc_text or not doc_map:
                return False
            
            # Convert keys to int
            self.documents = {int(k): v for k, v in doc_text.items() if v}
            
            # Handle doc_map as list or dict
            if isinstance(doc_map, list):
                self.doc_urls = {i: url for i, url in enumerate(doc_map) if url}
            else:
                self.doc_urls = {int(k): v for k, v in doc_map.items() if v}
            
            if inv_index:
                self.inverted_index = inv_index
            
            # Recreate chunks
            for doc_id, text in self.documents.items():
                self.chunks[doc_id] = nlp.chunk_text(text)
            
            # Rebuild doc lengths
            total_length = 0
            for doc_id, text in self.documents.items():
                tokens = nlp.preprocess(text)
                self.doc_lengths[doc_id] = len(tokens)
                total_length += len(tokens)
            self.avg_doc_length = total_length / max(len(self.documents), 1)
            
            return len(self.documents) > 0
            
        except Exception as e:
            print(f"Firebase load error: {e}")
            return False
    
    def _save_to_firebase(self) -> None:
        """Save index to Firebase."""
        try:
            doc_text = {str(k): v for k, v in self.documents.items()}
            doc_map = {str(k): v for k, v in self.doc_urls.items()}
            
            firebase.http_put(self.FIREBASE_DOCTEXT_PATH, doc_text)
            firebase.http_put(self.FIREBASE_DOCMAP_PATH, doc_map)
            firebase.http_put(self.FIREBASE_INDEX_PATH, self.inverted_index)
            
            print("✅ Index saved to Firebase")
        except Exception as e:
            print(f"Firebase save error: {e}")
    
    def _bm25_score(self, query_terms: List[str], doc_id: int) -> float:
        """Calculate BM25 score for a document."""
        k1, b = CONFIG.BM25_K1, CONFIG.BM25_B
        N = len(self.documents)
        doc_len = self.doc_lengths.get(doc_id, 1)
        
        score = 0.0
        doc_text = self.documents.get(doc_id, "")
        doc_tokens = nlp.preprocess(doc_text)
        tf_counter = Counter(doc_tokens)
        
        for term in query_terms:
            if term not in self.inverted_index:
                continue
            
            df = len(self.inverted_index[term])
            idf = math.log((N - df + 0.5) / (df + 0.5) + 1)
            tf = tf_counter.get(term, 0)
            
            numerator = tf * (k1 + 1)
            denominator = tf + k1 * (1 - b + b * (doc_len / self.avg_doc_length))
            
            score += idf * (numerator / denominator)
        
        return score
    
    def search(self, query: str, top_k: int = None) -> List[Dict]:
        """Hybrid search using TF-IDF + BM25."""
        if not self.is_loaded:
            return []
        
        top_k = top_k or CONFIG.RAG_TOP_K
        query_terms = nlp.preprocess(query)
        
        results = []
        
        # Method 1: BM25 document-level scoring
        bm25_scores = {}
        for doc_id in self.documents.keys():
            score = self._bm25_score(query_terms, doc_id)
            if score > 0:
                bm25_scores[doc_id] = score
        
        # Method 2: TF-IDF chunk-level scoring
        tfidf_scores = {}
        if self._tfidf_vectorizer and self._chunk_vectors is not None:
            query_vec = self._tfidf_vectorizer.transform([query])
            similarities = cosine_similarity(query_vec, self._chunk_vectors).flatten()
            
            # Aggregate by document (max chunk score per doc)
            for idx, sim in enumerate(similarities):
                if sim > 0:
                    doc_id = self.all_chunks[idx][0]
                    if doc_id not in tfidf_scores or sim > tfidf_scores[doc_id]:
                        tfidf_scores[doc_id] = float(sim)
        
        # Combine scores (normalize and weight)
        all_docs = set(bm25_scores.keys()) | set(tfidf_scores.keys())
        
        max_bm25 = max(bm25_scores.values()) if bm25_scores else 1
        max_tfidf = max(tfidf_scores.values()) if tfidf_scores else 1
        
        combined = []
        for doc_id in all_docs:
            bm25_norm = bm25_scores.get(doc_id, 0) / max_bm25
            tfidf_norm = tfidf_scores.get(doc_id, 0) / max_tfidf
            
            # Weighted combination (60% BM25, 40% TF-IDF)
            final_score = 0.6 * bm25_norm + 0.4 * tfidf_norm
            combined.append((doc_id, final_score, bm25_scores.get(doc_id, 0), tfidf_scores.get(doc_id, 0)))
        
        # Sort by combined score
        combined.sort(key=lambda x: x[1], reverse=True)
        
        # Build results
        for doc_id, combined_score, bm25, tfidf in combined[:top_k]:
            results.append({
                "doc_id": doc_id,
                "score": combined_score,
                "bm25_score": bm25,
                "tfidf_score": tfidf,
                "url": self.doc_urls.get(doc_id, ""),
                "title": self._get_doc_title(doc_id),
            })
        
        return results
    
    def search_chunks(self, query: str, top_k: int = 5) -> List[Dict]:
        """Search at chunk level for more precise retrieval."""
        if not self._tfidf_vectorizer or self._chunk_vectors is None:
            return []
        
        query_vec = self._tfidf_vectorizer.transform([query])
        similarities = cosine_similarity(query_vec, self._chunk_vectors).flatten()
        
        # Get top chunks
        top_indices = similarities.argsort()[-top_k:][::-1]
        
        results = []
        for idx in top_indices:
            if similarities[idx] > 0:
                doc_id, chunk_id, chunk_text = self.all_chunks[idx]
                results.append({
                    "doc_id": doc_id,
                    "chunk_id": chunk_id,
                    "score": float(similarities[idx]),
                    "text": chunk_text[:500],
                    "url": self.doc_urls.get(doc_id, ""),
                    "title": self._get_doc_title(doc_id),
                })
        
        return results
    
    def _get_doc_title(self, doc_id: int) -> str:
        """Extract title from document."""
        text = self.documents.get(doc_id, "")
        match = re.search(r"^TITLE:\s*(.+)$", text, re.MULTILINE)
        if match:
            return match.group(1).strip()[:100]
        return f"Document {doc_id}"
    
    def answer_question(self, question: str, top_k: int = 3) -> Dict:
        """Answer question using RAG with LLM."""
        # Search for relevant chunks
        chunks = self.search_chunks(question, top_k=top_k * 2)
        docs = self.search(question, top_k=top_k)
        
        if not chunks and not docs:
            return {
                "answer": "No relevant documents found for your question.",
                "sources": [],
                "chunks": []
            }
        
        # Build context from top chunks
        context_parts = []
        seen_chunks = set()
        for chunk in chunks[:5]:
            chunk_key = (chunk["doc_id"], chunk["chunk_id"])
            if chunk_key not in seen_chunks:
                seen_chunks.add(chunk_key)
                context_parts.append(f"[Source: {chunk['title']}]\n{chunk['text']}")
        
        context = "\n\n---\n\n".join(context_parts)
        
        # Generate answer with LLM
        prompt = f"""Based on the following context, answer the question. 
If the answer is not in the context, say "I don't have enough information to answer this."

Context:
{context}

Question: {question}

Answer:"""
        
        try:
            answer = AIClients.generate_gemini(prompt, temperature=0.3, max_tokens=512)
        except:
            answer = "Error generating answer. Please try again."
        
        return {
            "answer": answer,
            "sources": [d["url"] for d in docs if d["url"]],
            "chunks": chunks,
            "docs": docs
        }


# Create global RAG service
rag_service = RAGService()
print("✅ RAG service ready")


## 🎮 11. Gamification Service

In [ ]:
# =============================================================================
# GAMIFICATION SERVICE - Points, missions, rewards
# =============================================================================

class GamificationService:
    """Gamification system with Firebase persistence."""
    
    FIREBASE_PATH = "gamification/global"
    
    WHEEL_REWARDS = [
        ("+5 points", {"points": 5}),
        ("+10 points", {"points": 10}),
        ("+20 points", {"points": 20}),
        ("Coupon: 5% off", {"coupon": {"code": "CG-5OFF", "label": "5% off"}}),
        ("Coupon: 10% off", {"coupon": {"code": "CG-10OFF", "label": "10% off"}}),
        ("+50 points", {"points": 50}),
    ]
    
    DEFAULT_PROFILE = {
        "points": 0,
        "spins_available": 0,
        "missions": {
            "sync_data": {"last_completed": None, "total_completed": 0},
            "analyze_plant": {"last_completed": None, "total_completed": 0},
            "generate_report": {"last_completed": None, "total_completed": 0},
        },
        "coupons": []
    }
    
    @classmethod
    def _today_key(cls) -> str:
        return datetime.now(ZoneInfo(CONFIG.TIMEZONE)).strftime("%Y-%m-%d")
    
    @classmethod
    def _now_iso(cls) -> str:
        return datetime.now(ZoneInfo(CONFIG.TIMEZONE)).isoformat()
    
    @classmethod
    def get_profile(cls) -> Dict:
        """Get current gamification profile."""
        data = firebase.get(cls.FIREBASE_PATH) or {}
        
        profile = {
            "points": int(data.get("points", 0)),
            "spins_available": int(data.get("spins_available", 0)),
            "missions": data.get("missions", {}) or {},
            "coupons": data.get("coupons", []) or [],
        }
        
        # Merge default missions
        for mid, base in cls.DEFAULT_PROFILE["missions"].items():
            if mid not in profile["missions"]:
                profile["missions"][mid] = dict(base)
        
        return profile
    
    @classmethod
    def save_profile(cls, profile: Dict) -> None:
        """Save profile to Firebase."""
        firebase.set(cls.FIREBASE_PATH, profile)
    
    @classmethod
    def complete_mission(cls, mission_id: str, points: int) -> Tuple[Dict, bool]:
        """Complete a mission (once per day). Returns (profile, earned_today)."""
        profile = cls.get_profile()
        today = cls._today_key()
        
        mission = profile["missions"].get(mission_id, {"last_completed": None, "total_completed": 0})
        
        # Check if already completed today
        if mission.get("last_completed") == today:
            return profile, False
        
        # Award points and spin
        profile["points"] += points
        profile["spins_available"] += 1
        
        mission["last_completed"] = today
        mission["total_completed"] = mission.get("total_completed", 0) + 1
        profile["missions"][mission_id] = mission
        
        cls.save_profile(profile)
        return profile, True
    
    @classmethod
    def spin_wheel(cls) -> Tuple[Dict, str]:
        """Spin the wheel if spins available. Returns (profile, reward_message)."""
        profile = cls.get_profile()
        
        if profile["spins_available"] <= 0:
            return profile, "No spins available! Complete missions to earn spins."
        
        profile["spins_available"] -= 1
        
        # Random reward
        reward_label, reward_data = random.choice(cls.WHEEL_REWARDS)
        
        if "points" in reward_data:
            profile["points"] += reward_data["points"]
        
        if "coupon" in reward_data:
            coupon = {
                "code": f"{reward_data['coupon']['code']}-{random.randint(1000,9999)}",
                "label": reward_data['coupon']['label'],
                "created_at": cls._now_iso(),
                "redeemed": False
            }
            profile["coupons"].append(coupon)
        
        cls.save_profile(profile)
        return profile, f"🎉 You won: {reward_label}!"
    
    @classmethod
    def get_status_html(cls) -> str:
        """Get HTML status display."""
        profile = cls.get_profile()
        today = cls._today_key()
        
        missions_html = ""
        mission_names = {
            "sync_data": "Sync Data",
            "analyze_plant": "Analyze Plant",
            "generate_report": "Generate Report"
        }
        
        for mid, mdata in profile["missions"].items():
            completed_today = mdata.get("last_completed") == today
            status = "✅" if completed_today else "⬜"
            name = mission_names.get(mid, mid)
            missions_html += f"<div>{status} {name}</div>"
        
        return f"""
        <div style='padding:15px; background:#f0f8ff; border-radius:10px;'>
            <h3>🎮 Your Stats</h3>
            <p><b>Points:</b> {profile['points']}</p>
            <p><b>Spins Available:</b> {profile['spins_available']}</p>
            <h4>Daily Missions:</h4>
            {missions_html}
            <p><small>Complete missions to earn points & spins!</small></p>
        </div>
        """


# Create alias for backward compatibility
gamification = GamificationService()
print("✅ Gamification service ready")


## 📄 12. Report Generator

In [ ]:
# =============================================================================
# REPORT GENERATOR - Create DOCX reports
# =============================================================================

class ReportGenerator:
    """Generate DOCX reports from sensor data."""
    
    @staticmethod
    def unify_sensor_dfs(dfs: Dict) -> pd.DataFrame:
        """Unify multiple sensor dataframes into one."""
        def prep(df, col):
            if df is None or df.empty:
                return pd.DataFrame(columns=["timestamp", col])
            
            out = df.copy()
            
            # Normalize column names
            if "timestamp" not in out.columns and "created_at" in out.columns:
                out = out.rename(columns={"created_at": "timestamp"})
            
            if "timestamp" not in out.columns:
                if out.index.name:
                    out = out.reset_index()
                else:
                    out = out.reset_index().rename(columns={"index": "timestamp"})
            
            if "timestamp" not in out.columns or "value" not in out.columns:
                return pd.DataFrame(columns=["timestamp", col])
            
            out = out[["timestamp", "value"]]
            
            # Convert timestamp
            ts = out["timestamp"]
            if pd.api.types.is_numeric_dtype(ts):
                unit = "ms" if ts.median() > 1e12 else "s"
                out["timestamp"] = pd.to_datetime(ts, unit=unit, utc=True).dt.tz_convert(CONFIG.TIMEZONE).dt.tz_localize(None)
            else:
                out["timestamp"] = pd.to_datetime(ts, errors="coerce", utc=True).dt.tz_convert(CONFIG.TIMEZONE).dt.tz_localize(None)
            
            out = out.dropna(subset=["timestamp"])
            out["value"] = pd.to_numeric(out["value"], errors="coerce")
            out = out.dropna(subset=["value"])
            out = out.rename(columns={"value": col})
            
            return out
        
        t = prep(dfs.get("temperature"), "temperature")
        h = prep(dfs.get("humidity"), "humidity")
        s = prep(dfs.get("soil"), "soil")
        
        df = t.merge(h, on="timestamp", how="outer").merge(s, on="timestamp", how="outer")
        return df.sort_values("timestamp").reset_index(drop=True)
    
    @staticmethod
    def generate_ai_summary(df: pd.DataFrame) -> str:
        """Generate AI summary of sensor data."""
        if df.empty:
            return "No data available for analysis."
        
        # Get last 24 hours
        try:
            cutoff = df["timestamp"].max() - timedelta(hours=24)
            daily = df[df["timestamp"] >= cutoff]
        except:
            daily = df.tail(100)
        
        if daily.empty:
            daily = df.tail(50)
        
        # Calculate statistics
        stats = {}
        for col in ["temperature", "humidity", "soil"]:
            if col in daily.columns:
                stats[col] = {
                    "mean": daily[col].mean(),
                    "min": daily[col].min(),
                    "max": daily[col].max(),
                    "std": daily[col].std()
                }
        
        prompt = f"""Generate a brief daily summary for a smart garden monitoring system.

Sensor Statistics (last 24 hours):
- Temperature: avg={stats.get('temperature', {}).get('mean', 0):.1f}°C, range={stats.get('temperature', {}).get('min', 0):.1f}-{stats.get('temperature', {}).get('max', 0):.1f}°C
- Humidity: avg={stats.get('humidity', {}).get('mean', 0):.1f}%, range={stats.get('humidity', {}).get('min', 0):.1f}-{stats.get('humidity', {}).get('max', 0):.1f}%
- Soil Moisture: avg={stats.get('soil', {}).get('mean', 0):.1f}%, range={stats.get('soil', {}).get('min', 0):.1f}-{stats.get('soil', {}).get('max', 0):.1f}%

Healthy ranges: Temperature 18-32°C, Humidity 35-75%, Soil 20-60%

Write 2-3 sentences summarizing plant health status and any recommendations."""
        
        try:
            return AIClients.generate_cerebras(prompt)
        except:
            return "AI summary unavailable."
    
    @classmethod
    def create_docx(cls, dfs: Dict, limit: int = 50) -> str:
        """Create DOCX report file. Returns file path."""
        df = cls.unify_sensor_dfs(dfs)
        
        if df.empty:
            raise ValueError("No data available for report")
        
        doc = Document()
        
        # Title
        title = doc.add_heading("CloudGarden Daily Report", 0)
        title.alignment = WD_ALIGN_PARAGRAPH.CENTER
        
        # Date
        date_para = doc.add_paragraph()
        date_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
        date_para.add_run(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M')}").italic = True
        
        doc.add_paragraph()
        
        # AI Summary
        doc.add_heading("Executive Summary", level=1)
        summary = cls.generate_ai_summary(df)
        doc.add_paragraph(summary)
        
        # Statistics Table
        doc.add_heading("Environmental Conditions", level=1)
        
        table = doc.add_table(rows=1, cols=5)
        table.style = "Table Grid"
        
        headers = ["Metric", "Current", "Average", "Min", "Max"]
        for i, header in enumerate(headers):
            table.rows[0].cells[i].text = header
        
        for col, unit in [("temperature", "°C"), ("humidity", "%"), ("soil", "%")]:
            if col in df.columns:
                row = table.add_row()
                row.cells[0].text = col.capitalize()
                row.cells[1].text = f"{df[col].iloc[-1]:.1f}{unit}"
                row.cells[2].text = f"{df[col].mean():.1f}{unit}"
                row.cells[3].text = f"{df[col].min():.1f}{unit}"
                row.cells[4].text = f"{df[col].max():.1f}{unit}"
        
        # Save
        fd, path = tempfile.mkstemp(suffix=".docx")
        os.close(fd)
        doc.save(path)
        
        return path


print("✅ Report generator ready")


## 📊 13. Dashboard Functions

In [ ]:
# =============================================================================
# DASHBOARD FUNCTIONS - IoT visualization
# =============================================================================

def create_kpi_card(label: str, value: str, color: str, trend: str = "") -> str:
    """Create HTML KPI card."""
    trend_html = f"<span class='trend-{'up' if '↑' in trend else 'down'}'>{trend}</span>" if trend else ""
    return f"""
    <div class='kpi-card' style='border-left-color:{color};'>
        <div class='kpi-label'>{label}</div>
        <div class='kpi-value'>{value}</div>
        {trend_html}
    </div>
    """


def dashboard_screen():
    """Generate full dashboard visualizations."""
    df = sensor_service.load_from_firebase()
    
    if df.empty:
        empty_msg = "No data available. Please sync data first."
        return empty_msg, empty_msg, None, empty_msg, None, empty_msg, None, empty_msg, None, empty_msg, None
    
    # KPI Cards
    latest = df.iloc[-1]
    kpi_html = f"""
    <div style='display:flex; gap:20px; flex-wrap:wrap;'>
        {create_kpi_card('Temperature', f"{latest['temperature']:.1f}°C", CONFIG.COLOR_TEMP)}
        {create_kpi_card('Humidity', f"{latest['humidity']:.1f}%", CONFIG.COLOR_HUMIDITY)}
        {create_kpi_card('Soil Moisture', f"{latest['soil']:.1f}%", CONFIG.COLOR_SOIL)}
    </div>
    """
    
    # Stats
    stats_html = f"""
    <div style='display:flex; gap:20px; flex-wrap:wrap;'>
        <div style='padding:15px; background:#f5f5f5; border-radius:8px; flex:1;'>
            <b>Temperature</b><br>
            Avg: {df['temperature'].mean():.1f}°C | 
            Min: {df['temperature'].min():.1f}°C | 
            Max: {df['temperature'].max():.1f}°C
        </div>
        <div style='padding:15px; background:#f5f5f5; border-radius:8px; flex:1;'>
            <b>Humidity</b><br>
            Avg: {df['humidity'].mean():.1f}% | 
            Min: {df['humidity'].min():.1f}% | 
            Max: {df['humidity'].max():.1f}%
        </div>
        <div style='padding:15px; background:#f5f5f5; border-radius:8px; flex:1;'>
            <b>Soil</b><br>
            Avg: {df['soil'].mean():.1f}% | 
            Min: {df['soil'].min():.1f}% | 
            Max: {df['soil'].max():.1f}%
        </div>
    </div>
    """
    
    # Time Series Plot
    fig_ts = make_subplots(rows=3, cols=1, shared_xaxes=True, subplot_titles=("Temperature", "Humidity", "Soil Moisture"))
    fig_ts.add_trace(go.Scatter(x=df['timestamp'], y=df['temperature'], name='Temp', line=dict(color=CONFIG.COLOR_TEMP)), row=1, col=1)
    fig_ts.add_trace(go.Scatter(x=df['timestamp'], y=df['humidity'], name='Humidity', line=dict(color=CONFIG.COLOR_HUMIDITY)), row=2, col=1)
    fig_ts.add_trace(go.Scatter(x=df['timestamp'], y=df['soil'], name='Soil', line=dict(color=CONFIG.COLOR_SOIL)), row=3, col=1)
    fig_ts.update_layout(height=600, showlegend=False)
    
    # Correlation
    corr = df[['temperature', 'humidity', 'soil']].corr()
    fig_corr = px.imshow(corr, text_auto='.2f', color_continuous_scale='RdBu_r', title="Correlation Matrix")
    corr_html = "<p>Shows relationships between sensor readings</p>"
    
    # Hourly patterns
    df['hour'] = df['timestamp'].dt.hour
    hourly = df.groupby('hour')[['temperature', 'humidity', 'soil']].mean()
    fig_hourly = go.Figure()
    fig_hourly.add_trace(go.Bar(x=hourly.index, y=hourly['temperature'], name='Temp'))
    fig_hourly.add_trace(go.Bar(x=hourly.index, y=hourly['humidity'], name='Humidity'))
    fig_hourly.update_layout(title="Average by Hour", barmode='group')
    hourly_html = "<p>Average sensor values by hour of day</p>"
    
    # Daily trends
    df['date'] = df['timestamp'].dt.date
    daily = df.groupby('date')[['temperature', 'humidity', 'soil']].mean()
    fig_daily = go.Figure()
    for col, color in [('temperature', CONFIG.COLOR_TEMP), ('humidity', CONFIG.COLOR_HUMIDITY), ('soil', CONFIG.COLOR_SOIL)]:
        fig_daily.add_trace(go.Scatter(x=daily.index, y=daily[col], name=col.capitalize(), line=dict(color=color)))
    fig_daily.update_layout(title="Daily Averages")
    daily_html = "<p>Daily average trends</p>"
    
    # Distribution
    fig_dist = make_subplots(rows=1, cols=3, subplot_titles=("Temperature", "Humidity", "Soil"))
    fig_dist.add_trace(go.Histogram(x=df['temperature'], nbinsx=20, marker_color=CONFIG.COLOR_TEMP), row=1, col=1)
    fig_dist.add_trace(go.Histogram(x=df['humidity'], nbinsx=20, marker_color=CONFIG.COLOR_HUMIDITY), row=1, col=2)
    fig_dist.add_trace(go.Histogram(x=df['soil'], nbinsx=20, marker_color=CONFIG.COLOR_SOIL), row=1, col=3)
    fig_dist.update_layout(height=300, showlegend=False)
    dist_html = "<p>Distribution of sensor readings</p>"
    
    return kpi_html, stats_html, fig_ts, corr_html, fig_corr, hourly_html, fig_hourly, daily_html, fig_daily, dist_html, fig_dist


def dashboard_moving_avg(variable: str):
    """Calculate moving averages."""
    df = sensor_service.load_from_firebase()
    
    if df.empty or variable not in df.columns:
        return "No data", None
    
    df['MA_7'] = df[variable].rolling(window=7, min_periods=1).mean()
    df['MA_24'] = df[variable].rolling(window=24, min_periods=1).mean()
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['timestamp'], y=df[variable], name='Raw', opacity=0.5))
    fig.add_trace(go.Scatter(x=df['timestamp'], y=df['MA_7'], name='MA-7'))
    fig.add_trace(go.Scatter(x=df['timestamp'], y=df['MA_24'], name='MA-24'))
    fig.update_layout(title=f"{variable.capitalize()} Moving Averages")
    
    return f"Moving averages for {variable}", fig


print("✅ Dashboard functions ready")


## 🌿 14. Plant Analysis Functions

In [ ]:
# =============================================================================
# PLANT ANALYSIS - Disease detection and recommendations
# =============================================================================

def analyze_plant_image(image, temp: float, humidity: float, soil: float):
    """Analyze plant image and sensor data."""
    # Run image classification
    preds = plant_classifier(image)
    top = preds[0]
    label = top["label"]
    score = top["score"]
    
    # Check sensor conditions
    alerts = []
    advice = []
    
    # Temperature checks
    if temp < CONFIG.TEMP_MIN:
        alerts.append("Low temperature")
        advice.append("Move plant to a warmer environment")
    elif temp > CONFIG.TEMP_MAX:
        alerts.append("High temperature")
        advice.append("Move plant to a shaded area")
    
    # Humidity checks
    if humidity < CONFIG.HUMIDITY_MIN:
        alerts.append("Low air humidity")
        advice.append("Increase humidity (misting)")
    elif humidity > CONFIG.HUMIDITY_MAX:
        alerts.append("High air humidity")
        advice.append("Improve ventilation")
    
    # Soil checks
    if soil < CONFIG.SOIL_MIN:
        alerts.append("Low soil moisture")
        advice.append("Water the plant")
    elif soil > CONFIG.SOIL_MAX:
        alerts.append("High soil moisture")
        advice.append("Reduce watering")
    
    # Determine status based on image
    is_bad = "healthy" not in label.lower()
    
    status_html = f"""
    <div style='padding:15px; border-radius:10px; 
         background:{'#ffdddd' if is_bad else '#ddffdd'};
         border:2px solid {'#ff0000' if is_bad else '#00aa00'};
         font-weight:bold; font-size:18px;'>
        {'🔴 Plant Status: NEEDS ATTENTION' if is_bad else '🟢 Plant Status: HEALTHY'}
    </div>
    """
    
    if not alerts:
        alerts.append("All sensor readings normal")
    
    return (
        f"Detected: {label} ({score:.1%} confidence)",
        status_html,
        "\n".join(f"⚠️ {a}" for a in alerts),
        "\n".join(f"💡 {a}" for a in advice) if advice else "No immediate actions needed."
    )


# Gamified wrapper
def analyze_plant_gamified(image, temp, humidity, soil):
    """Analyze plant with gamification reward."""
    result = analyze_plant_image(image, temp, humidity, soil)
    
    # Award mission completion
    profile, earned = GamificationService.complete_mission("analyze_plant", 15)
    if earned:
        print(f"🎮 +15 points! Total: {profile['points']}")
    
    return result


print("✅ Plant analysis functions ready")


## 🎯 15. Gamified Action Wrappers

In [ ]:
# =============================================================================
# GAMIFIED WRAPPERS - Add points to actions
# =============================================================================

def sync_screen_gamified():
    """Sync data with gamification."""
    msg, count = sensor_service.sync_to_firebase()
    
    if count > 0:
        profile, earned = GamificationService.complete_mission("sync_data", 10)
        if earned:
            msg += f"\n\n🎮 Mission Complete! +10 points (Total: {profile['points']})"
    
    return msg


def generate_report_screen_gamified(samples: int):
    """Generate report with gamification."""
    try:
        # Fetch sensor data
        dfs = {
            "temperature": sensor_service.fetch_from_server("temperature", samples),
            "humidity": sensor_service.fetch_from_server("humidity", samples),
            "soil": sensor_service.fetch_from_server("soil", samples),
        }
        
        # Generate report
        path = ReportGenerator.create_docx(dfs, samples)
        
        # Award mission
        profile, earned = GamificationService.complete_mission("generate_report", 20)
        
        status = f"✅ Report generated successfully!"
        if earned:
            status += f"\n🎮 Mission Complete! +20 points (Total: {profile['points']})"
        
        return status, path
        
    except Exception as e:
        return f"❌ Error: {e}", None


print("✅ Gamified wrappers ready")


## 🔧 16. Initialize RAG Index

In [ ]:
# =============================================================================
# INITIALIZE RAG INDEX
# =============================================================================

print("Loading RAG index...")
rag_service.build_index(CONFIG.DOC_URLS)
print(f"✅ RAG ready with {len(rag_service.documents)} documents")


## 🎨 17. UI Tab Builders

In [ ]:
# =============================================================================
# UI TAB BUILDERS - Gradio interface components
# =============================================================================

def build_realtime_dashboard_tab():
    """Build the realtime dashboard tab."""
    gr.Markdown("### 🌿 Real-Time Plant Status")
    
    samples = gr.Slider(1, 200, value=20, step=1, label="Number of Samples")
    btn = gr.Button("Update Dashboard", variant="primary")
    
    status = gr.Textbox(label="Status", lines=1)
    details = gr.Textbox(label="Details", lines=4)
    
    with gr.Row():
        gr.Markdown(f"""
        <div style='padding:14px; border:1px solid #ddd; border-radius:10px;'>
            <h4>🌿 Status Legend</h4>
            <span style='color:{CONFIG.COLOR_OK}; font-size:20px;'>●</span> Healthy - All normal<br>
            <span style='color:{CONFIG.COLOR_WARN}; font-size:20px;'>●</span> Warning - Near limits<br>
            <span style='color:{CONFIG.COLOR_BAD}; font-size:20px;'>●</span> Not OK - Out of range
        </div>
        """)
        
        gr.Markdown(f"""
        <div style='padding:14px; border:1px solid #ddd; border-radius:10px;'>
            <h4>ℹ️ Valid Ranges</h4>
            <span style='color:{CONFIG.COLOR_TEMP}; font-size:20px;'>●</span> Temperature: {CONFIG.TEMP_MIN}-{CONFIG.TEMP_MAX}°C<br>
            <span style='color:{CONFIG.COLOR_HUMIDITY}; font-size:20px;'>●</span> Humidity: {CONFIG.HUMIDITY_MIN}-{CONFIG.HUMIDITY_MAX}%<br>
            <span style='color:{CONFIG.COLOR_SOIL}; font-size:20px;'>●</span> Soil: {CONFIG.SOIL_MIN}-{CONFIG.SOIL_MAX}%
        </div>
        """)
    
    gr.Markdown("### 📈 Sensor Graphs")
    
    with gr.Row():
        plot_temp = gr.Plot(label="Temperature")
        plot_hum = gr.Plot(label="Humidity")
    
    with gr.Row():
        plot_soil = gr.Plot(label="Soil Moisture")
        plot_combined = gr.Plot(label="Combined")
    
    def update_dashboard(limit):
        dfs = {
            "temperature": sensor_service.fetch_from_server("temperature", limit),
            "humidity": sensor_service.fetch_from_server("humidity", limit),
            "soil": sensor_service.fetch_from_server("soil", limit),
        }
        
        missing = [k for k, v in dfs.items() if v is None]
        if missing:
            return "⚠️ Partial Data", f"Missing: {', '.join(missing)}", None, None, None, None
        
        temp_val = float(dfs["temperature"]["value"].iloc[-1])
        hum_val = float(dfs["humidity"]["value"].iloc[-1])
        soil_val = float(dfs["soil"]["value"].iloc[-1])
        
        status_text, color, details_list = sensor_service.check_plant_status(temp_val, hum_val, soil_val)
        
        details_text = f"{'; '.join(details_list)}\nLatest: temp={temp_val:.1f}, hum={hum_val:.1f}, soil={soil_val:.1f}"
        
        # Create plots
        fig_t = plt.figure(figsize=(7, 3))
        plt.plot(dfs["temperature"]["created_at"], dfs["temperature"]["value"], color=CONFIG.COLOR_TEMP)
        plt.title("Temperature")
        plt.grid(True)
        plt.close()
        
        fig_h = plt.figure(figsize=(7, 3))
        plt.plot(dfs["humidity"]["created_at"], dfs["humidity"]["value"], color=CONFIG.COLOR_HUMIDITY)
        plt.title("Humidity")
        plt.grid(True)
        plt.close()
        
        fig_s = plt.figure(figsize=(7, 3))
        plt.plot(dfs["soil"]["created_at"], dfs["soil"]["value"], color=CONFIG.COLOR_SOIL)
        plt.title("Soil Moisture")
        plt.grid(True)
        plt.close()
        
        # Combined normalized
        fig_c = plt.figure(figsize=(7, 3))
        for name, df, color in [("Temp", dfs["temperature"], CONFIG.COLOR_TEMP),
                                 ("Humidity", dfs["humidity"], CONFIG.COLOR_HUMIDITY),
                                 ("Soil", dfs["soil"], CONFIG.COLOR_SOIL)]:
            vals = df["value"]
            normalized = (vals - vals.min()) / (vals.max() - vals.min() + 1e-8)
            plt.plot(df["created_at"], normalized, label=name, color=color)
        plt.legend()
        plt.title("Combined (Normalized)")
        plt.grid(True)
        plt.close()
        
        return status_text, details_text, fig_t, fig_h, fig_s, fig_c
    
    btn.click(update_dashboard, inputs=[samples], outputs=[status, details, plot_temp, plot_hum, plot_soil, plot_combined])


def build_iot_dashboard_tab():
    """Build comprehensive IoT analytics dashboard."""
    gr.Markdown("### 📈 Comprehensive Sensor Analytics")
    
    refresh_btn = gr.Button("🔄 Refresh All Data", variant="primary")
    
    gr.Markdown("#### 📌 Current Readings")
    kpi_html = gr.HTML()
    
    gr.Markdown("#### 📊 Statistics")
    stats_html = gr.HTML()
    
    gr.Markdown("#### 📈 Time Series")
    ts_plot = gr.Plot()
    
    gr.Markdown("#### 🔗 Correlation")
    corr_card = gr.HTML()
    corr_plot = gr.Plot()
    
    gr.Markdown("#### ⏰ Hourly Patterns")
    hourly_card = gr.HTML()
    hourly_plot = gr.Plot()
    
    gr.Markdown("#### 📅 Daily Trends")
    daily_card = gr.HTML()
    daily_plot = gr.Plot()
    
    gr.Markdown("#### 📊 Distribution")
    dist_card = gr.HTML()
    dist_plot = gr.Plot()
    
    refresh_btn.click(
        dashboard_screen,
        outputs=[kpi_html, stats_html, ts_plot, corr_card, corr_plot, hourly_card, hourly_plot, daily_card, daily_plot, dist_card, dist_plot]
    )
    
    gr.Markdown("#### 📉 Moving Averages")
    with gr.Row():
        ma_var = gr.Dropdown(choices=['temperature', 'humidity', 'soil'], value='temperature', label='Variable')
        ma_btn = gr.Button("Calculate")
    ma_card = gr.HTML()
    ma_plot = gr.Plot()
    
    ma_btn.click(dashboard_moving_avg, inputs=[ma_var], outputs=[ma_card, ma_plot])


def build_sync_data_tab():
    """Build data sync tab."""
    gr.Markdown("### 🔄 Sync Data from Server")
    gr.Markdown("Upload IoT sensor data to Firebase database")
    
    sync_btn = gr.Button("🔄 Sync New Data", variant="primary")
    sync_output = gr.Textbox(label="Status", lines=5)
    
    sync_btn.click(sync_screen_gamified, outputs=[sync_output])


def build_generate_report_tab():
    """Build report generation tab."""
    gr.Markdown("### 📄 Generate Report")
    gr.Markdown("Create a DOCX report with sensor data analysis")
    
    samples = gr.Slider(5, 200, value=20, step=1, label="Samples per sensor")
    btn = gr.Button("📥 Generate Report", variant="primary")
    status = gr.Textbox(label="Status", lines=2)
    file = gr.File(label="Download")
    
    btn.click(generate_report_screen_gamified, inputs=[samples], outputs=[status, file])


def build_plant_disease_tab():
    """Build plant disease detection tab."""
    gr.Markdown("### 🖼️ Plant Disease Detection")
    
    with gr.Row():
        with gr.Column(scale=2):
            image = gr.Image(type="filepath", label="Upload Plant Image")
            temp = gr.Slider(0, 45, value=25, label="Temperature (°C)")
            humidity = gr.Slider(0, 100, value=50, label="Humidity (%)")
            soil = gr.Slider(0, 100, value=50, label="Soil Moisture (%)")
            btn = gr.Button("Analyze Plant", variant="primary")
        
        with gr.Column(scale=2):
            diagnosis = gr.Textbox(label="Diagnosis")
            status = gr.HTML(label="Status")
            alerts = gr.Textbox(label="Alerts", lines=4)
            recommendations = gr.Textbox(label="Recommendations", lines=4)
    
    btn.click(analyze_plant_gamified, inputs=[image, temp, humidity, soil], outputs=[diagnosis, status, alerts, recommendations])


print("✅ UI builders ready (part 1)")


## 🎨 18. UI Tab Builders (Part 2)

In [ ]:
# =============================================================================
# UI TAB BUILDERS - RAG, Chat, Rewards
# =============================================================================

def build_rag_chat_tab():
    """Build RAG-powered Q&A tab."""
    gr.Markdown("### 🔍 RAG Q&A - Ask about Plant Science")
    gr.Markdown("Ask questions about the indexed research papers")
    
    with gr.Row():
        question = gr.Textbox(label="Question", placeholder="Ask about plant diseases, soil health, etc.", lines=2)
    
    with gr.Row():
        top_k = gr.Slider(1, 5, value=3, step=1, label="Top-K documents")
    
    ask_btn = gr.Button("🔍 Search & Answer", variant="primary")
    
    answer = gr.Textbox(label="Answer", lines=6)
    sources = gr.Textbox(label="Sources & Ranking", lines=10)
    
    def rag_search(q, k):
        if not q.strip():
            return "Please enter a question.", ""
        
        result = rag_service.answer_question(q, top_k=k)
        
        # Format sources
        source_lines = []
        if result.get("docs"):
            source_lines.append("📊 Document Ranking:")
            for d in result["docs"]:
                source_lines.append(f"  • Doc {d['doc_id']}: score={d['score']:.4f} | {d['title'][:50]}")
        
        if result.get("sources"):
            source_lines.append("\n📎 Sources:")
            for url in result["sources"]:
                source_lines.append(f"  • {url}")
        
        if result.get("chunks"):
            source_lines.append(f"\n📄 Used {len(result['chunks'])} relevant chunks")
        
        return result.get("answer", "No answer found."), "\n".join(source_lines)
    
    ask_btn.click(rag_search, inputs=[question, top_k], outputs=[answer, sources])


def build_gemini_chat_tab():
    """Build Gemini chat tab with database persistence."""
    gr.Markdown("### 💬 AI Chat (with History)")
    gr.Markdown("Chat with AI - conversations are saved to database")
    
    # Session management
    with gr.Row():
        session_dropdown = gr.Dropdown(choices=[], label="Load Session", scale=3)
        new_session_btn = gr.Button("New Session", scale=1)
        refresh_sessions_btn = gr.Button("🔄", scale=1)
    
    session_id_display = gr.Textbox(label="Current Session", interactive=False)
    
    chat = gr.Chatbot(label="Chat History", height=400)
    state = gr.State([])
    
    msg = gr.Textbox(label="Message", placeholder="Type your message...", lines=2)
    temperature = gr.Slider(0.0, 1.0, value=0.7, step=0.05, label="Creativity")
    
    with gr.Row():
        send_btn = gr.Button("Send", variant="primary")
        clear_btn = gr.Button("Clear Session")
    
    def refresh_sessions():
        sessions = chat_service.get_all_sessions()
        choices = [f"{s['id']} ({s['message_count']} msgs)" for s in sessions]
        return gr.update(choices=choices)
    
    def new_session():
        session_id = chat_service.start_new_session()
        return session_id, [], []
    
    def load_session(selection):
        if not selection:
            return "", [], []
        session_id = selection.split(" (")[0]
        chat_service.load_session(session_id)
        return session_id, chat_service.session_history, chat_service.session_history
    
    def send_message(message, history, temp):
        if not message.strip():
            return "", history, history
        
        answer, new_history = chat_service.chat(message, temp)
        return "", new_history, new_history
    
    def clear_chat():
        new_history = chat_service.clear_session()
        return chat_service.current_session_id, [], []
    
    # Wire up events
    refresh_sessions_btn.click(refresh_sessions, outputs=[session_dropdown])
    new_session_btn.click(new_session, outputs=[session_id_display, chat, state])
    session_dropdown.change(load_session, inputs=[session_dropdown], outputs=[session_id_display, chat, state])
    send_btn.click(send_message, inputs=[msg, state, temperature], outputs=[msg, chat, state])
    msg.submit(send_message, inputs=[msg, state, temperature], outputs=[msg, chat, state])
    clear_btn.click(clear_chat, outputs=[session_id_display, chat, state])


def build_rewards_tab():
    """Build gamification rewards tab."""
    profile = GamificationService.get_profile()
    
    gr.Markdown("### 🎮 Farm Rewards")
    gr.Markdown("Complete missions to earn points and spins!")
    
    with gr.Row():
        points_box = gr.Textbox(label="Points", value=str(profile["points"]), interactive=False)
        spins_box = gr.Textbox(label="Spins Available", value=str(profile["spins_available"]), interactive=False)
    
    status_html = gr.HTML(value=GamificationService.get_status_html())
    
    spin_btn = gr.Button("🎰 Spin the Wheel!", variant="primary")
    spin_result = gr.Textbox(label="Result", lines=2)
    
    coupons_md = gr.Markdown("### Your Coupons")
    coupons_list = gr.Textbox(label="Coupons", lines=4, value="No coupons yet")
    
    refresh_btn = gr.Button("🔄 Refresh Stats")
    
    def spin():
        profile, msg = GamificationService.spin_wheel()
        coupons_text = "\n".join([f"• {c['code']} - {c['label']}" for c in profile.get("coupons", [])]) or "No coupons yet"
        return str(profile["points"]), str(profile["spins_available"]), msg, coupons_text, GamificationService.get_status_html()
    
    def refresh():
        profile = GamificationService.get_profile()
        coupons_text = "\n".join([f"• {c['code']} - {c['label']}" for c in profile.get("coupons", [])]) or "No coupons yet"
        return str(profile["points"]), str(profile["spins_available"]), coupons_text, GamificationService.get_status_html()
    
    spin_btn.click(spin, outputs=[points_box, spins_box, spin_result, coupons_list, status_html])
    refresh_btn.click(refresh, outputs=[points_box, spins_box, coupons_list, status_html])


print("✅ UI builders ready (part 2)")


## 🎨 19. Custom CSS

In [ ]:
# =============================================================================
# CUSTOM CSS
# =============================================================================

CUSTOM_CSS = """
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap');

* { font-family: 'Inter', sans-serif; }

.kpi-card {
    background: white;
    padding: 24px;
    border-radius: 12px;
    box-shadow: 0 1px 3px rgba(0,0,0,0.12);
    text-align: center;
    border-left: 4px solid;
    min-width: 150px;
}

.kpi-label { color: #6b7280; font-size: 14px; font-weight: 600; }
.kpi-value { font-size: 36px; font-weight: 700; color: #1f2937; }
.trend-up { color: #10b981; }
.trend-down { color: #ef4444; }

.gradio-container { max-width: 1400px !important; }
"""

print("✅ CSS ready")


## 🚀 20. Launch Application

In [ ]:
# =============================================================================
# MAIN APPLICATION - Build and launch Gradio app
# =============================================================================

def build_app():
    """Build the complete Gradio application."""
    
    with gr.Blocks(css=CUSTOM_CSS, title="CloudGarden") as app:
        
        gr.Markdown("""
        # 🌱 CloudGarden
        ### Smart Plant Monitoring & Disease Detection System
        """)
        
        with gr.Tabs():
            with gr.Tab("🌿 Realtime Dashboard"):
                build_realtime_dashboard_tab()
            
            with gr.Tab("📊 IoT Analytics"):
                build_iot_dashboard_tab()
            
            with gr.Tab("🔄 Sync Data"):
                build_sync_data_tab()
            
            with gr.Tab("📄 Generate Report"):
                build_generate_report_tab()
            
            with gr.Tab("🖼️ Disease Detection"):
                build_plant_disease_tab()
            
            with gr.Tab("🔍 RAG Q&A"):
                build_rag_chat_tab()
            
            with gr.Tab("💬 AI Chat"):
                build_gemini_chat_tab()
            
            with gr.Tab("🎮 Rewards"):
                build_rewards_tab()
    
    return app


# Build and launch
print("\n" + "="*50)
print("🚀 Starting CloudGarden Application...")
print("="*50 + "\n")

app = build_app()
app.launch(share=True, debug=False)
